In [11]:
using Plots
using CSV
using DataFrames
using Statistics
using Printf

dados = CSV.read("dados2/ALL0002/F0002CH1.CSV", DataFrame)

tempo = dados[!, 4]
voltagem = dados[!, 5]

plot(tempo, voltagem, label="DDP", xlabel="Tempo (s)", ylabel="DDP (V)", title="Gráfico de DDP vs Tempo - Sem o Núcleo", legend=:topright, grid=true, linewidth=2, color=:seagreen)

# Save the plot as a PNG file
savefig("latex/figuras/grafico_diferenca_potencial - 2.png")

"/home/pedro/Desktop/exp3.6/latex/figuras/grafico_diferenca_potencial - 2.png"

In [12]:
function trapezio(x, y, x0, xN, y_min)
    mask = (x .>= x0) .& (x .<= xN) .& (y .>= y_min)
    if sum(mask) < 3
        return 0.0
    end

    xf = x[mask]
    yf = y[mask]
    dx = xf[2] - xf[1]
    integral = (yf[1] + yf[end]) * dx / 2
    integral += sum(yf[2:end-1]) * dx
    return integral
end

trapezio (generic function with 1 method)

In [14]:
N = 10000  # Número de voltas da bobina
area = 2.5e-3
comprimento = 7.0e-2


function integra_diretorio(diretorio; fator_v=1.0, x0=0, xN=1, y_min=-0.1)
    integrals = Float64[]
    for (root, _, files) in walkdir(diretorio)
        for f in files
            if endswith(f, ".CSV")
                path = joinpath(root, f)
                df   = CSV.read(path, DataFrame)
                t    = df[!, 4] # tempo (já em segundos não precisa converter)
                v    = df[!, 5] .* fator_v # aplica fator de escala

                I = trapezio(t, v, x0, xN, y_min * fator_v)
                push!(integrals, I)
                #println("[$(basename(root))/$(f)]: Integral = $(round(I, digits=5))")
            end
        end
    end
    return integrals
end

# Função para calcular estatísticas básicas
function calc_stats(dados, N)
    fluxo = dados / N
    media = mean(fluxo)
    desvio = std(fluxo)
    incerteza = desvio / sqrt(length(fluxo))
    return media, desvio, incerteza
end

# Função para imprimir resultados formatados
function print_results(nome, media, desvio, incerteza)
    @printf "%s: Média = %.3e, Desvio = %.3e, Incerteza = %.3e\n" nome media desvio incerteza
end

# Função para calcular campo magnético
function calc_campo(media, incerteza, area)
    B = media / area
    B_err = incerteza / area
    return B, B_err
end

# Processa todos os conjuntos de dados
N = 10000
area = 2.5e-3
mu_0 = 4 * pi * 1e-7

# Calcula estatísticas para todos os dados
stats = []
for (i, dados) in enumerate([dados1, dados2, dados3])
    push!(stats, calc_stats(dados, N))
end

# Exibe resultados
nomes = ["Fluxo 1 (Imã)", "Fluxo 2 (Sem Núcleo)", "Fluxo 3 (Núcleo Ferro)"]
println("Resultados:")
for (i, nome) in enumerate(nomes)
    print_results(nome, stats[i]...)
end

# Cálculos específicos
media2, _, fluxo2_err = stats[2]
media3, _, fluxo3_err = stats[3]

# Permeabilidade e incerteza
mu_max = ((media3 + fluxo3_err) / (media2 - fluxo2_err)) * mu_0
mu_min = ((media3 - fluxo3_err) / (media2 + fluxo2_err)) * mu_0
mu_bar = (media3 / media2) * mu_0
delta_mu = (mu_max - mu_min) / 2

@printf "Permeabilidade Relativa = %.2f\n" (media3/media2)
@printf "μ = %.3e ± %.3e T·m/A\n" mu_bar delta_mu

# Campos magnéticos
for (i, nome) in enumerate(["B_imã", "B_ar", "B_ferro"])
    B, B_err = calc_campo(stats[i][1], stats[i][3], area)
    @printf "%s = %.3e ± %.3e T\n" nome B B_err
end


Resultados:
Fluxo 1 (Imã): Média = 9.578e-05, Desvio = 3.888e-07, Incerteza = 1.039e-07
Fluxo 2 (Sem Núcleo): Média = 2.710e-08, Desvio = 9.338e-10, Incerteza = 2.696e-10
Fluxo 3 (Núcleo Ferro): Média = 5.605e-07, Desvio = 1.651e-08, Incerteza = 4.579e-09
Permeabilidade Relativa = 20.68
μ = 2.599e-05 ± 4.708e-07 T·m/A
B_imã = 3.831e-02 ± 4.157e-05 T
B_ar = 1.084e-05 ± 1.078e-07 T
B_ferro = 2.242e-04 ± 1.832e-06 T


In [80]:
# Mesmo cálculo, mas usando Interpolations e QuadGK
using Interpolations, QuadGK

x0, xN, y_min = 0, 1, -0.1

for (root, _, files) in walkdir("dados3")
    for f in files
        if endswith(f, ".CSV")
            path = joinpath(root, f)
            df   = CSV.read(path, DataFrame)
            t    = df[!, 4]
            v    = df[!, 5]

            itp = LinearInterpolation(t, v; extrapolation_bc=0)

            f_int(x) = (val = itp(x); val >= y_min ? val : 0)
            I, err = quadgk(f_int, x0, xN)

            println("[$(basename(root))/$(f)]: Integral = $(round(I, digits=4)) ± $(round(err, digits=11))")
        end
    end
end

[ALL0000/F0000CH1.CSV]: Integral = 0.0287 ± 4.3e-10
[ALL0001/F0001CH1.CSV]: Integral = 0.0283 ± 4.2e-10
[ALL0002/F0002CH1.CSV]: Integral = 0.0289 ± 4.3e-10
[ALL0003/F0003CH1.CSV]: Integral = 0.0288 ± 4.3e-10
[ALL0004/F0004CH1.CSV]: Integral = 0.0278 ± 4.1e-10
[ALL0005/F0005CH1.CSV]: Integral = 0.027 ± 4.0e-10
[ALL0006/F0006CH1.CSV]: Integral = 0.0273 ± 4.1e-10
[ALL0007/F0007CH1.CSV]: Integral = 0.0276 ± 4.1e-10
[ALL0008/F0008CH1.CSV]: Integral = 0.0276 ± 4.1e-10
[ALL0009/F0009CH1.CSV]: Integral = 0.0275 ± 4.1e-10
[ALL0010/F0010CH1.CSV]: Integral = 0.0276 ± 4.1e-10
[ALL0011/F0011CH1.CSV]: Integral = 0.0273 ± 4.1e-10
[ALL0012/F0012CH1.CSV]: Integral = 0.0299 ± 4.4e-10


In [ ]:
using Plots, CSV, DataFrames, Printf

# Mapeamento dos títulos para cada pasta
titulos = Dict(
    "dados1" => "Imã",
    "dados2" => "Sem Núcleo",
    "dados3" => "Núcleo de Ferro"
)

# Lista para armazenar comandos LaTeX das figuras
figuras_latex = String[]

# Gera os gráficos e prepara LaTeX
for pasta in ["dados1", "dados2", "dados3"]
    for (root, _, files) in walkdir(pasta)
        for f in files
            if endswith(f, ".CSV")
                path = joinpath(root, f)
                dados = CSV.read(path, DataFrame)
                tempo = dados[!, 4]
                voltagem = dados[!, 5]

                # Título e nome do arquivo
                titulo = titulos[pasta]
                nome_base = @sprintf "grafico_%s_%s.png" basename(pasta) splitext(f)[1]
                caminho_fig = "latex/figuras/$(nome_base)"

                # Gera e salva o gráfico
                plot(tempo, voltagem, label="DDP", xlabel="Tempo (s)", ylabel="DDP (V)",
                     title="Gráfico de DDP vs Tempo - $titulo",
                     legend=:topright, grid=true, linewidth=2, color=:seagreen)
                savefig(caminho_fig)

                # Adiciona entrada LaTeX SEM caption
                push!(figuras_latex, """
\\begin{figure}[H]
    \\centering
    \\includegraphics[width=0.45\\linewidth]{$(caminho_fig)}
\\end{figure}
""")
            end
        end
    end
end

# Gera o arquivo LaTeX para ser incluído via \\input em outro .tex
open("latex/graficos.tex", "w") do io
    println(io, """
\\subsection*{Apêndice - Gráficos de DDP vs Tempo}
\\begin{center}
\\begin{multicols}{2}
""")
    for fig in figuras_latex
        println(io, fig)
    end
    println(io, """
\\end{multicols}
\\end{center}
""")
end